In [1]:
import numpy as np 
import pandas as pd
import sklearn
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
df = pd.read_csv('spam.csv', encoding='ISO-8859-1') 


In [2]:
dataframe=df.iloc[:,0:2]

In [3]:
dataframe.rename(columns={'v1':'label','v2':'message'},inplace=True)

C:\Users\Rahul\AppData\Local\Temp\ipykernel_19200\3288895809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.rename(columns={'v1':'label','v2':'message'},inplace=True)


In [4]:
dataframe['label'] = dataframe['label'].map({'ham': 0, 'spam': 1})

C:\Users\Rahul\AppData\Local\Temp\ipykernel_19200\1692912076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['label'] = dataframe['label'].map({'ham': 0, 'spam': 1})


In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer ,WordNetLemmatizer

In [6]:
nltk.download('punkt')       # Primary tokenizer
nltk.download('punkt_tab')  
dataframe['tokens'] = dataframe['message'].apply(word_tokenize)
print(dataframe['tokens'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


0       [Go, until, jurong, point, ,, crazy, .., Avail...
1                [Ok, lar, ..., Joking, wif, u, oni, ...]
2       [Free, entry, in, 2, a, wkly, comp, to, win, F...
3       [U, dun, say, so, early, hor, ..., U, c, alrea...
4       [Nah, I, do, n't, think, he, goes, to, usf, ,,...
                              ...                        
5567    [This, is, the, 2nd, time, we, have, tried, 2,...
5568     [Will, Ì_, b, going, to, esplanade, fr, home, ?]
5569    [Pity, ,, *, was, in, mood, for, that, ., So, ...
5570    [The, guy, did, some, bitching, but, I, acted,...
5571                  [Rofl, ., Its, true, to, its, name]
Name: tokens, Length: 5572, dtype: object


C:\Users\Rahul\AppData\Local\Temp\ipykernel_19200\1657836277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['tokens'] = dataframe['message'].apply(word_tokenize)


In [7]:
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))
dataframe['filtered_tokens'] = dataframe['tokens'].apply(
    lambda tokens: [token.lower() for token in tokens if token.lower() not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Rahul\AppData\Local\Temp\ipykernel_19200\3843461603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['filtered_tokens'] = dataframe['tokens'].apply(


In [8]:
print(dataframe['filtered_tokens'])

0       [go, jurong, point, ,, crazy, .., available, b...
1                [ok, lar, ..., joking, wif, u, oni, ...]
2       [free, entry, 2, wkly, comp, win, fa, cup, fin...
3       [u, dun, say, early, hor, ..., u, c, already, ...
4       [nah, n't, think, goes, usf, ,, lives, around,...
                              ...                        
5567    [2nd, time, tried, 2, contact, u., u, å£750, p...
5568               [ì_, b, going, esplanade, fr, home, ?]
5569           [pity, ,, *, mood, ., ..., suggestions, ?]
5570    [guy, bitching, acted, like, 'd, interested, b...
5571                                [rofl, ., true, name]
Name: filtered_tokens, Length: 5572, dtype: object


In [9]:
stemmer=SnowballStemmer('english')
lemmatizer=WordNetLemmatizer()

In [10]:
dataframe['stemmed_tokens'] = dataframe['filtered_tokens'].apply(
    lambda tokens: [stemmer.stem(token) for token in tokens]
)

In [11]:
dataframe['lemmatized_tokens']=dataframe['filtered_tokens'].apply(
    lambda tokens: [stemmer.stem(token) for token in tokens])

In [12]:
from gensim.models import KeyedVectors
w2vmodel = KeyedVectors.load_word2vec_format(
    'C:/Users/Rahul/Downloads/GoogleNews-vectors-negative300.bin.gz',
    binary=True
)

In [13]:
def get_average_vector(tokens, model):
    valid_vectors = [model[word] for word in tokens if word in model.key_to_index]
    if valid_vectors:
        return np.mean(valid_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  
dataframe['vector'] = dataframe['stemmed_tokens'].apply(lambda tokens: get_average_vector(tokens, model))


NameError: name 'model' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

X = np.vstack(dataframe['vector'].values)  
y = dataframe['label'].values              

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('X_train:',X_train)
print('X_test:',X_test)
print('y_train:',y_train)
print('y_test:',y_test)

In [ ]:
clff= LogisticRegression(max_iter=1000)  
clff.fit(X_train, y_train)
y_pred = clff.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
def predict_message_class(model,w2vmodel, message):
    import numpy as np
    import re
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem import SnowballStemmer

    stop_words = set(stopwords.words('english'))
    stemmer = SnowballStemmer("english")
    tokens = word_tokenize(message)
    filtered = [stemmer.stem(w) for w in tokens if w not in stop_words]
    valid_vectors = [w2vmodel[word] for word in filtered if word in w2vmodel.key_to_index]
    if valid_vectors:
        vector = np.mean(valid_vectors, axis=0).reshape(1, -1)
    else:
        vector = np.zeros(w2vmodel.vector_size).reshape(1, -1)

    # Predict using the trained classifier
    prediction = clff.predict(vector)[0]
    return prediction



In [ ]:
message = "Congratulations You've won a free ticket. Text WIN to 12345."
print(predict_message_class(model=dataframe, w2vmodel=model, message=message))



# question 2


In [ ]:
!pip install contractions
!pip install emoji

In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import contractions
import emoji

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

df = pd.read_csv('Tweets.csv.zip')  
df1 = df[['text', 'airline_sentiment']]

lemmatizer = WordNetLemmatizer()

def preprocess_tweet(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = contractions.fix(text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [token for token in tokens if not emoji.is_emoji(token)]
    return ' '.join(tokens)

df1['clean_text'] = df1['text'].apply(preprocess_tweet)


In [ ]:
from gensim.models import KeyedVectors
w2vmodel = KeyedVectors.load_word2vec_format(
    'C:/Users/Rahul/Downloads/GoogleNews-vectors-negative300.bin.gz',
    binary=True
)

In [ ]:
import numpy as np

def tweet_to_vec(tweet, model, dim=300):
    words = tweet.split()
    valid_words = [w for w in words if w in model]
    if not valid_words:
        return np.zeros(dim)
    return np.mean([model[w] for w in valid_words], axis=0)

X = np.vstack(df1['clean_text'].apply(lambda x: tweet_to_vec(x, w2vmodel)).values)
y = df1['airline_sentiment'].values


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.4f}")


In [ ]:
def predict_tweet_sentiment(model, w2v_model, tweet):
    clean = preprocess_tweet(tweet)
    vec = tweet_to_vec(clean, w2v_model)
    pred = model.predict([vec])[0]
    return pred

# Example 
tweet = "I love flying with Delta, always a great experience!"
print(predict_tweet_sentiment(clf, w2vmodel, tweet))
